# LangChain Tool, Tool Calling, Agent 튜토리얼

이 노트북은 LangChain의 핵심 개념인 **Tool**, **Tool Calling**, **Agent**를 단계별로 학습할 수 있도록 구성되었습니다.

## 📚 학습 목차

1. **Tool 기초** - 첫 번째 Tool 만들기
2. **Tool Calling** - LLM이 Tool을 호출하는 방법
3. **Agent** - 복잡한 문제를 단계별로 해결하기

---

## 🎯 학습 목표

이 노트북을 완료하면:
- Tool을 직접 만들 수 있습니다
- LLM이 Tool을 자동으로 사용하도록 설정할 수 있습니다  
- 복잡한 업무를 처리하는 Agent를 개발할 수 있습니다

---

## ⚙️ 환경 설정

먼저 필요한 패키지를 설치하고 API 키를 설정해야 합니다.


In [2]:
# 필요한 패키지 설치 (주석 해제해서 실행)
# !pip install langchain langchain-openai python-dotenv

# 패키지 import
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Annotated
import random
from datetime import datetime

# API 키 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 패키지 로드 완료!")
print("⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!")
print("   (.env 파일에 OPENAI_API_KEY=your_key_here)")


✅ 패키지 로드 완료!
⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!
   (.env 파일에 OPENAI_API_KEY=your_key_here)


# 1️⃣ Tool 기초

## Tool이란?
**Tool**은 LLM(AI 모델)이 특정 작업을 수행할 수 있도록 해주는 함수입니다.

예를 들어:
- 🧮 계산기 역할을 하는 Tool
- 🔍 웹 검색을 하는 Tool  
- 📁 파일을 읽는 Tool
- 💾 데이터베이스를 조회하는 Tool

LLM은 텍스트만 생성할 수 있지만, Tool을 통해 실제 작업을 수행할 수 있게 됩니다!

## 첫 번째 Tool 만들기


In [3]:
# 가장 기본적인 Tool 만들기
@tool
def simple_calculator(
    a: Annotated[int, "첫 번째 숫자"], 
    b: Annotated[int, "두 번째 숫자"]
) -> int:
    """두 숫자를 더하는 간단한 계산기입니다."""
    result = a + b
    print(f"계산 결과: {a} + {b} = {result}")
    return result

# Tool의 정보 확인
print("=== Tool 정보 ===")
print(f"Tool 이름: {simple_calculator.name}")
print(f"Tool 설명: {simple_calculator.description}")
print(f"Tool 파라미터: {simple_calculator.args}")
print()

# Tool 직접 실행해보기
print("=== Tool 실행 ===")
result = simple_calculator.invoke({"a": 5, "b": 3}) ## 사람이 직접 Tool 호출 -> Tool의 기능 실행
print(f"직접 실행 결과: {result}")


=== Tool 정보 ===
Tool 이름: simple_calculator
Tool 설명: 두 숫자를 더하는 간단한 계산기입니다.
Tool 파라미터: {'a': {'description': '첫 번째 숫자', 'title': 'A', 'type': 'integer'}, 'b': {'description': '두 번째 숫자', 'title': 'B', 'type': 'integer'}}

=== Tool 실행 ===
계산 결과: 5 + 3 = 8
직접 실행 결과: 8


## 더 복잡한 Tool들 만들기

이제 좀 더 실용적인 Tool들을 만들어봅시다.


In [4]:
# 텍스트 분석 Tool
@tool
def text_analyzer(text: Annotated[str, "분석할 텍스트"]) -> dict:
    """텍스트를 분석해서 다양한 정보를 반환합니다."""
    
    words = text.split()
    word_count = len(words)
    char_count = len(text)
    char_count_no_space = len(text.replace(" ", ""))
    longest_word = max(words, key=len) if words else ""
    
    result = {
        "단어_개수": word_count,
        "문자_개수": char_count,
        "공백_제외_문자_개수": char_count_no_space,
        "가장_긴_단어": longest_word,
        "가장_긴_단어_길이": len(longest_word)
    }
    
    return result

# 가짜 날씨 정보 Tool
@tool
def weather_info(city: Annotated[str, "도시 이름"]) -> str:
    """가짜 날씨 정보를 제공합니다 (실제 API 연결 없음)."""
    
    temperatures = [15, 18, 22, 25, 28, 30, 12, 8]
    weather_conditions = ["맑음", "흐림", "비", "눈", "안개"]
    
    temp = random.choice(temperatures)
    condition = random.choice(weather_conditions)
    
    return f"{city}의 현재 날씨: {condition}, 기온: {temp}°C"

# Tool들 테스트
print("=== 텍스트 분석 Tool 테스트 ===")
sample_text = "안녕하세요! LangChain Tool을 배우고 있습니다. 정말 재미있어요!"
analysis_result = text_analyzer.invoke({"text": sample_text})
print(f"분석 결과: {analysis_result}")

print("\n=== 날씨 정보 Tool 테스트 ===")
weather_result = weather_info.invoke({"city": "서울"})
print(f"날씨 정보: {weather_result}")

# Tool 목록 관리
my_tools = [simple_calculator, text_analyzer, weather_info]
print(f"\n=== 사용 가능한 Tool 목록 ===")
for i, tool in enumerate(my_tools, 1):
    print(f"{i}. {tool.name}: {tool.description}")


=== 텍스트 분석 Tool 테스트 ===
분석 결과: {'단어_개수': 7, '문자_개수': 42, '공백_제외_문자_개수': 36, '가장_긴_단어': 'LangChain', '가장_긴_단어_길이': 9}

=== 날씨 정보 Tool 테스트 ===
날씨 정보: 서울의 현재 날씨: 흐림, 기온: 30°C

=== 사용 가능한 Tool 목록 ===
1. simple_calculator: 두 숫자를 더하는 간단한 계산기입니다.
2. text_analyzer: 텍스트를 분석해서 다양한 정보를 반환합니다.
3. weather_info: 가짜 날씨 정보를 제공합니다 (실제 API 연결 없음).


## 🎯 Tool 핵심 포인트

- ✅ `@tool` 데코레이터로 일반 함수를 Tool로 변환
- ✅ `Annotated`로 파라미터 설명 추가
- ✅ docstring으로 Tool 기능 설명
- ✅ `tool.invoke()`로 직접 실행 가능
- ✅ 여러 Tool을 리스트로 관리